## 演習
- 以下を実装してください.
  - 最初はPytorch
  - 余力があればnumpyで
- 実装するもの
  - クロスエントロピー
  - ソフトマックス回帰の仮説
- 条件
  - パラメーター$W$を3×3行列
  - 入力、出力ともに三次元に固

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### 回答

In [20]:
W = torch.rand(3, 3, requires_grad=True)
x = torch.tensor([[1., 2., 3.]])
# 仮説 
nn.Softmax(F.linear(x, W))

Softmax(dim=tensor([[2.3139, 2.4260, 4.3950]], grad_fn=<MmBackward>))

In [37]:
criterion = nn.CrossEntropyLoss() # 損失の定義
print(criterion(F.linear(x, W), torch.tensor([1])))

## Softmax回帰の出力＋普通の損失
criterion2 = nn.NLLLoss()
o = nn.Softmax(dim=1)((F.linear(x, W)))
a = torch.tensor([1])
criterion2(torch.log(o), a)

tensor(2.2036, grad_fn=<NllLossBackward>)


tensor(2.2036, grad_fn=<NllLossBackward>)

In [30]:
o

Softmax(dim=tensor([[2.3139, 2.4260, 4.3950]], grad_fn=<MmBackward>))

## 演習問題
- 上ではデータが一つの場合のクロスエントロピーを計算した
- データが$D = (x_1, y_1), \ldots (x_N, y_N)$の場合のクロスエントロピーを計算せ

### 回答
$$
\frac{1}{N}  \left( \sum_{\ell = 1}^N  -  q^{(\ell)}_j x^{(\ell)}_i +  \frac{x^{(\ell)}_i \exp(\sum_{i=1}^n w_{ji}x^{(\ell)}_i)}{\sum_{k=1}^m \exp(\sum_{i=1}^n w_{ki}x^{(\ell)}_i)}  \right)
$$

## 演習問題
- ロジスティック回帰のクロスエントロピーの微分を計算せよ

### 回答

$$
\left(\sum_{k=1}^N x_{ki}(y_k - \sigma(wx_k)), \ldots, \sum_{k=1}^N x_{kn}(y_k - \sigma(wx_k)\right)
$$

## 確認の演習問題
- シグモイド関数を微分せよ.
- $x_1, x_2 \in \mathbb{R}^2$ とし $y_1, y_2 \in  \{0, 1\}$とする.
  - パラメータ$w$を二次元のベクトルとして,ロジスティック回帰の式を記述せよ
  - ロジスティック回帰のクロスエントロピーを記述せよ
  - $x_1 = (2, 3)$, $x_2 = (3, 1)$, $y_1 = 1, y_2 = 0$とした時,クロスエントロピーのヤコビ行列を計算せよ

### 回答
- シグモイド関数の微分は$\sigma(x)(1 - \sigma(x))$

In [60]:
w = torch.rand(2, requires_grad=True)
x1 = torch.tensor([2., 4.], requires_grad=True)
x2 = torch.tensor([3., 1.], requires_grad=True)
y1 = torch.tensor([1])
y2 = torch.tensor([0])

- ロジスティック回帰: $\sigma(wx)$

In [61]:
o = y1 * torch.log(torch.sigmoid(w.dot(x1)))  + (1 -y1) * torch.log(1 - torch.sigmoid(w.dot(x1)))

In [62]:
o += y2 * torch.log(torch.sigmoid(w.dot(x2)))  + (1 -y2) * torch.log(1 - torch.sigmoid(w.dot(x2)))

In [63]:
o

tensor([-1.2899], grad_fn=<AddBackward0>)

In [65]:
torch.autograd.grad(o, [x1, x2], retain_graph=True)

(tensor([0.0106, 0.0492]), tensor([-0.0746, -0.3472]))


## 演習
- Sofmax回帰を実装
  - pytorchを使ってSoftmax回帰をしてください
    - データはirisを使う.
  - (時間があれば)sklearnを使いSoftmax回帰をする.
  - (時間があれば)sklearnの乳がんのデータを使い,自作のロジスティック回帰で実装しよう.
- よければ評価データと訓練データに対し,Accuracyを測定しよう
  - 二値ではないので,今回はAccuracyのみ

### 回答

In [66]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

# データの分割
X_train, X_valid, y_train, y_valid = train_test_split(iris.data, iris.target, test_size=0.2)

In [67]:
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, index):
        
        data = self.data[index]
        label = self.labels[index]
        
        if not self.transform is None:
            data, label = self.transform(data, label)
        
        return data, label

In [68]:
def iris_transform(data, label):
    # inputはdoubleをfloatにしている.
    return torch.tensor(data).float(), torch.tensor(label)

iris = load_iris()
X_train, X_valid, y_train, y_valid = train_test_split(iris.data, iris.target, test_size=0.2)

iris_train_dataset = IrisDataset(X_train, y_train, iris_transform)
iris_valid_dataset = IrisDataset(X_valid, y_valid, iris_transform)

In [69]:
batch_size  = 24 # ミニバッチのデータの数
iris_train_dataloader = torch.utils.data.DataLoader(iris_train_dataset, batch_size=batch_size, shuffle=True)
iris_valid_dataloader = torch.utils.data.DataLoader(iris_valid_dataset, batch_size=batch_size, shuffle=True)

In [70]:
model = nn.Linear(4, 3)

max_epoch = 100 #

criterion = nn.CrossEntropyLoss() # 損失の定義
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #(確率的)勾配降下法

In [136]:
model.train()
for data, target in iris_train_dataloader: # 入力と正解
    optimizer.zero_grad() # 勾配の初期化
    output = model(data) # 仮説で値代入
    loss = criterion(output, target) # 損失
    loss.backward() # 微分の計算
    optimizer.step() # パラメータの更新

In [137]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

pred = output.argmax(axis=1).cpu() # 正解予測のラベル取得
pred = pred.detach().numpy() # numpyに変換
ans = target.cpu().numpy() 
# numpyにしなくても動作する.
print("accuracy", accuracy_score(ans, pred)) # 他も一緒だが

accuracy 0.875
